In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

rng = np.random.default_rng(seed=1234)


@dataclass
class Data:
    n: int
    m: int
    X: np.ndarray
    y: np.ndarray
    D: np.ndarray


@dataclass
class GDResult:
    topt: np.ndarray
    kopt: int
    k: int
    Jmin: float
    Jval: np.ndarray

    def print_info(self):
        print("topt: ", self.topt)
        print(f"kopt / ktotal: {self.kopt} / {self.k}")
        print(f"Jmin: {self.Jmin}")

def load():
    X = np.loadtxt("nuclear_x.csv", delimiter=",")
    (n, m) = X.shape

    y = np.loadtxt("nuclear_y.csv", delimiter=",")[..., np.newaxis]

    D = np.hstack((X, y))

    rng.shuffle(D, axis=0)

    X = np.hstack((np.ones((n, 1)), D[:, :m]))
    y = D[:, m:]

    return Data(n, m, X, y, D)


data = load()

n, m = (data.n, data.m)
X, y  = (data.X, data.y)

Pw = np.diag((0,) + (1,) * m)

M = y * X
MT = M.T

l = 1e-3

t0 = rng.standard_normal((3, 1))
t0[0, 0] = 0.0

In [ ]:
def J(t):
    v = 1 - M @ t
    v *= v > 0
    return np.squeeze(l / 2 * t.T @ Pw @ t + np.average(v))

def G(t):
    return l * Pw @ t - 1 / n * MT @ (1 - M @ t > 0)

class SG:
    def __init__(self, n, m, D):
        self.n, self.m = (n,m)
        self.D = np.hstack((np.ones((self.n, 1)), D))
        self.i = self.n
        self.rng = np.random.default_rng(seed=1234)
        self.c = 0
    
    def __call__(self, t):
        if self.i == self.n:
            self.rng.shuffle(self.D)
            self.i = 0
        xi = self.D[self.i:self.i+1, :self.m+1]
        yi = self.D[self.i, self.m+1]
        self.i += 1
        self.c+=1
        print(f"{self.c}\r", end="")
        gr = yi / n * xi.T if np.squeeze(yi * xi @ t) < 1 else 0
        return l / n * Pw @ t - gr 

def GD(t0, G, ak, stop=-np.inf, Kmax=10000):
    t = np.copy(t0)

    kopt = 0
    topt = np.zeros_like(t)
    
    Jk = J(t)
    Jmin = Jk
    Jval = [Jk]

    for k in range(1, Kmax + 1):
        t -= ak(k) * G(t)
        Jk = J(t)
        if Jk < Jmin:
            kopt = k
            Jmin = Jk
            np.copyto(src=t, dst=topt)
        Jval.append(Jk)
        if Jk < stop:
            break

    return GDResult(
        topt=np.squeeze(topt),
        kopt=kopt,
        k=k,
        Jmin=Jmin,
        Jval=Jval,
    )

def plot_classes(ax, data, topt):
    D = data.D
    m = data.m
    class1 = D[D[:, m] == 1][:, :m].T
    class2 = D[D[:, m] == -1][:, :m].T
    x = D[:, 0]
    xmin, xmax = (np.min(x), np.max(x))
    x = np.linspace(xmin, xmax)
    [t0, t1, t2] = topt

    ax.scatter(class1[0], class1[1], label="Class 1", color="blue")
    ax.scatter(class2[0], class2[1], label="Class 2", color="red")
    ax.plot(x, -1 / t2 * (t1 * x + t0), color="black", label="Decision Boundary")
    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")
    ax.legend()
    return ax

In [ ]:
gdres = GD(t0, G, ak=lambda k: 100 / k, stop=0.3, Kmax=10000)

In [ ]:
fig, [ax1, ax2] = plt.subplots(1,2, constrained_layout=True, figsize=(8.3, 5.8))
fig.get_layout_engine().set(w_pad=12 / 72)

ax2.scatter(np.arange(gdres.k+1), np.log10(gdres.Jval), marker="x", s=1., color="blue")
ax2.set_xlabel(r"$k$")
ax2.set_ylabel(r"$\log_{10} J(\theta_k)$")

plot_classes(ax1, data, gdres.topt)
gdres.print_info()

In [ ]:
sgdres = GD(t0, SG(data.n, data.m, data.D), ak=lambda k: 100 / (1 + int(k / data.n)), stop=0.3, Kmax=10000 * data.n)


In [ ]:
fig, axs = plt.subplot_mosaic([
    ["ax1", "ax2"],
    ["ax3", "ax3"],
], layout="constrained", figsize=(8.3, 8.3))
fig.get_layout_engine().set(w_pad=12 / 72)

ax1 = axs["ax1"]
ax2 = axs["ax2"]
ax3 = axs["ax3"]

ax2.scatter(np.arange(sgdres.k+1)[:1000], np.log10(sgdres.Jval)[:1000], marker="x", s=1., color="blue")
ax2.set_xlabel(r"$k$")
ax2.set_ylabel(r"$\log_{10} J(\theta_k)$")

ax3.scatter(np.arange(sgdres.k+1), np.log10(sgdres.Jval), marker="x", s=1., color="blue")
ax3.set_xlabel(r"$k$")
ax3.set_ylabel(r"$\log_{10} J(\theta_k)$")

plot_classes(ax1, data, sgdres.topt)
sgdres.print_info()